In [1]:
import pandas as pd
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Load and preprocess data
df = pd.read_csv('Q&ADataset.csv')
questions = df['question'].values
answers = df['answer'].values
marks = df['marks'].values

In [3]:
df.head()

question  \
0                              What is Deep learning   
1  How do artificial neural networks function in ...   
2  What distinguishes deep learning from traditio...   
3  Explain the concept of supervised learning in ...   
4  What challenges are associated with deep learn...   

                                              answer  marks  
0  Deep learning is a subfield of machine learnin...      5  
1  Artificial neural networks (ANNs) in deep lear...      5  
2  Deep learning distinguishes itself from tradit...      5  
3  Supervised learning in deep learning is a para...      6  
4  Deep learning faces several challenges that im...      5

In [4]:
df.tail()

question  \
406  What is the role of sentiment analysis in info...   
407  Explain the concept of probabilistic informati...   
408  How do information retrieval systems adapt to ...   
409  Discuss the impact of mobile computing on info...   
410  What are the recent advancements and trends in...   

                                                answer  marks  
406  Sentiment analysis plays a pivotal role in inf...      5  
407  Probabilistic Information Retrieval (IR) model...      5  
408  Information retrieval systems continually adap...      5  
409  Mobile computing has profoundly transformed in...      5  
410  Recent advancements in information retrieval (...      5

In [5]:
# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
questions_seq = tokenizer.texts_to_sequences(questions)
answers_seq = tokenizer.texts_to_sequences(answers)

In [6]:
# Pad sequences to have the same length
max_len = 272  # Set the desired maximum length
max_len = max(max(len(seq) for seq in questions_seq), max(len(seq) for seq in answers_seq))
questions_seq = pad_sequences(questions_seq, maxlen=max_len)
answers_seq = pad_sequences(answers_seq, maxlen=max_len)

In [7]:
# Concatenate the two arrays along the second axis
X_combined = np.concatenate([questions_seq, answers_seq], axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, marks, test_size=0.2, random_state=42
)

In [ ]:
# Split training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [ ]:
# Modify the Embedding layer in your model definition
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_len*2))  # Multiply by 2 for concatenated input
model.add(LSTM(128))
model.add(Dense(1, activation='linear'))  # Assuming marks are continuous values


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])  # Using Mean Absolute Error as an additional metric


In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10, batch_size=32,
    validation_data=(X_val, y_val)
)

In [ ]:
# Plot training and validation metrics
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test set
loss, mae = model.evaluate([X_test], y_test)
print(f'Mean Squared Error on test set: {loss}')
print(f'Mean Absolute Error on test set: {mae}')

In [ ]:
# Make predictions on the test set
predictions = model.predict([X_test])

In [ ]:
# Generate confusion matrix
threshold = 0.5  # You may need to adjust this threshold based on your specific problem
binary_predictions = (predictions > threshold).astype(int)
conf_matrix = confusion_matrix(y_test > threshold, binary_predictions)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Take a single input and predict the output
def predict_single_input(question, answer):
    input_question = tokenizer.texts_to_sequences([question])
    input_answer = tokenizer.texts_to_sequences([answer])
    input_question = pad_sequences(input_question, maxlen=max_len)
    input_answer = pad_sequences(input_answer, maxlen=max_len)

    prediction = model.predict([input_question, input_answer])
    return prediction[0][0]

In [ ]:
# Example usage
example_question = "What is Deep learning"
example_answer = "Deep learning is a subfield of machine learning that focuses on the use of artificial neural networks to model and solve complex problems. It involves training deep neural networks, which are composed of multiple layers (deep architectures), to learn hierarchical representations of data. These networks are inspired by the structure and function of the human brain.Deep learning has shown remarkable success in various domains, such as computer vision, natural language processing, speech recognition, and reinforcement learning. Convolutional Neural Networks (CNNs) are commonly used for image-related tasks, while Recurrent Neural Networks (RNNs) are employed for sequence-based data, and Transformer models have shown excellence in natural language processing tasks.The depth of the neural network, achieved by having multiple hidden layers, allows these models to capture intricate patterns and representations, making them well-suited for tasks with large amounts of complex data. Deep learning has been a driving force behind many recent advancements in artificial intelligence and has demonstrated state-of-the-art performance in various applications"
output_prediction = predict_single_input(example_question, example_answer)
print(f'Predicted Marks: {output_prediction}')